# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [173]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [175]:
# 2) seu código aqui
qtd_missings_por_coluna = sinasc.isna().sum()
qtd_missings_por_coluna

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [177]:
# 3) seu código aqui
#Selecionar colunas indicadas
sinasc_col_filtered = sinasc.loc[:,sinasc.columns.isin(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 
                                     'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 
                                     'GRAVIDEZ', 'CONSULTAS', 'APGAR5'])].copy()

sinasc_col_filtered.info()

#Refazer a contagem de dados missing
sinasc_col_filtered.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     27028 non-null  int64  
 1   IDADEMAE    27028 non-null  int64  
 2   ESTCIVMAE   26711 non-null  float64
 3   ESCMAE      26716 non-null  object 
 4   QTDFILVIVO  25455 non-null  float64
 5   GESTACAO    25796 non-null  object 
 6   GRAVIDEZ    26949 non-null  object 
 7   CONSULTAS   27028 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.9+ MB


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [179]:
# 4) seu código aqui
#Contar número de missings na variável de interesse APGAR5
sinasc_col_filtered['APGAR5'].isna().sum()

#Remover os dados missings da coluna APGAR5
sinasc_clean_apgar5 = sinasc_col_filtered.dropna(subset=['APGAR5']).copy()

#Conte quantos missings existem na coluna APGAR5
print(f'Total de missings na coluna APGAR5: {sinasc_clean_apgar5['APGAR5'].isna().sum()}')

#Conte o número de linhas
print(f'Novo número de linhas e colunas no dataframe: {sinasc_clean_apgar5.shape}')

#Conte quantos missings existem no dataframe
print(f'\nValores missings restantes em cada variável:\n{sinasc_clean_apgar5.isna().sum()}')


Total de missings na coluna APGAR5: 0
Novo número de linhas e colunas no dataframe: (26925, 9)

Valores missings restantes em cada variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [181]:
# 5) Observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. 
#    Vamos assumir que o não preenchido é o mesmo que o código 9.
sinasc_clean_apgar5['ESTCIVMAE'] = sinasc_clean_apgar5['ESTCIVMAE'].fillna(9)
sinasc_clean_apgar5['CONSULTAS'] = sinasc_clean_apgar5['CONSULTAS'].fillna(9) #Esse dataset já estava limpo na coluna CONSULTAS


sinasc_clean_apgar5.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [183]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero
sinasc_clean_apgar5['QTDFILVIVO'] = sinasc_clean_apgar5['QTDFILVIVO'].fillna(0)

In [185]:
# 7) Das restantes, decida que valor te parece mais adequado 
#   (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. 
#   Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a 
#   todo o momento - não há necessariamente certo e errado aqui.

#Como se refere ao nível educacional, pode ter sido uma pergunta opcional no formulário e foi ignorado
sinasc_clean_apgar5['ESCMAE'] = sinasc_clean_apgar5['ESCMAE'].fillna(9) 

#A coluna GESTACAO  refere-se as semanas de gestação, como as opções englobam todas as quantidade de semanas é mais provável que tenha sido ignorado
sinasc_clean_apgar5['GESTACAO'] = sinasc_clean_apgar5['GESTACAO'].fillna(9) 

#A coluna GRAVIDEZ indica o tipo de gravidez, e como se trata de um banco de dados sobre nascimentos, independentemente se houve o nascimento ou não, 
#houve uma gravidez de um feto ou mais, então é provável que tenha sido ignorado
sinasc_clean_apgar5['GRAVIDEZ'] = sinasc_clean_apgar5['GRAVIDEZ'].fillna(9) 


In [187]:
# 8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:

#Entre 8 e 10 está em uma faixa 'normal'.
#Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
#Entre 4 e 5 significa 'asfixia moderada'.
#Entre 0 e 3 significa 'asfixia severa'.
#Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.


#Crie uma categorização dessa variável com essa codificação
sinasc_clean_apgar5.loc[sinasc_clean_apgar5['APGAR5'] <= 10, 'APGAR_CLASSIFICADO' ] = 'normal'
sinasc_clean_apgar5.loc[sinasc_clean_apgar5['APGAR5'] <= 7, 'APGAR_CLASSIFICADO' ] = 'asfixia leve'
sinasc_clean_apgar5.loc[sinasc_clean_apgar5['APGAR5'] <= 5, 'APGAR_CLASSIFICADO' ] = 'asfixia moderada'
sinasc_clean_apgar5.loc[sinasc_clean_apgar5['APGAR5'] <= 3, 'APGAR_CLASSIFICADO' ] = 'asfixia severa'

#calcule as frequências dessa categorização
sinasc_clean_apgar5['APGAR_CLASSIFICADO'].value_counts()

APGAR_CLASSIFICADO
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [189]:
sinasc_clean_apgar5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26925 entries, 0 to 27027
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LOCNASC             26925 non-null  int64  
 1   IDADEMAE            26925 non-null  int64  
 2   ESTCIVMAE           26925 non-null  float64
 3   ESCMAE              26925 non-null  object 
 4   QTDFILVIVO          26925 non-null  float64
 5   GESTACAO            26925 non-null  object 
 6   GRAVIDEZ            26925 non-null  object 
 7   CONSULTAS           26925 non-null  int64  
 8   APGAR5              26925 non-null  float64
 9   APGAR_CLASSIFICADO  26925 non-null  object 
dtypes: float64(3), int64(3), object(4)
memory usage: 2.3+ MB


In [191]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 
#    Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

sinasc_clean_apgar5.columns = (['local_nasc', 'idade_mae', 'estado_civil_mae', 'escolaridade_mae', 
                               'qtd_filhos_vivo', 'gestacao', 'gravidez', 'consultas', 'apgar_5','apgar_5_classificado'])


#como observado em aula para renomear colunas não é necessário o inplace = True, apenas no caso de renomear o index
sinasc_clean_apgar5



,local_nasc,idade_mae,estado_civil_mae,escolaridade_mae,qtd_filhos_vivo,gestacao,gravidez,consultas,apgar_5,apgar_5_classificado
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
